In [4]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3
import pandas as pd
import requests
from datetime import datetime
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [5]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [6]:
session = boto3.Session(profile_name='master')

In [7]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [8]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [9]:
params = {
    'vs_currency' : "usd",
    'days' : "1"
}
res = json.loads(call_get(API_ENDPOINT / "coins" / 'bitcoin' / 'ohlc', params))

In [10]:
df = pd.DataFrame(res, columns=['timestamp', 'open', 'high', 'low', 'close'])
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df['coin'] = 'bitcoin'
df['formated_date'] = df['datetime'].dt.strftime('%Y-%m-%d')

In [64]:
r2 = json.loads(call_get(API_ENDPOINT / "coins" / 'bitcoin' / "market_chart", {"vs_currency" : "usd", "days" : "max"}))

In [85]:
def transform_json_dataframe(data: dict):
    ts = [i[0] for i in data['prices']]
    prices = [i[1] for i in data['prices']]
    market_caps = [i[1] for i in data['market_caps']]
    total_volumes = [i[1] for i in data['total_volumes']]
    temp = pd.DataFrame({
        "timestamp" : ts,
        "prices" : prices,
        "market_caps" : market_caps,
        "total_volumes" : total_volumes,
        "formated_date" : [datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in ts]
        })
    temp['coin'] = 'bitcoin'
    temp['currency'] = 'usd'
    return temp


In [87]:
transform_json_dataframe(r2).dtypes

timestamp          int64
prices           float64
market_caps      float64
total_volumes    float64
formated_date     object
coin              object
currency          object
dtype: object

In [123]:
from boto3.dynamodb.conditions import Key
dyn = session.resource('dynamodb')
def query_pathes(type, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('Pathes')
    response = table.query(KeyConditionExpression="type = :good", ExpressionAttributeNames={"#dynobase_good": "good"})
    return response['Items']

In [124]:
query_pathes('good', dyn)

ClientError: An error occurred (ValidationException) when calling the Query operation: Invalid KeyConditionExpression: Attribute name is a reserved keyword; reserved keyword: type

In [12]:
def currencies_info(data: dict, list_of_keys: List[str]):
    filtered_json = {key: data[key] for key in list_of_keys}
    temp_df = pd.json_normalize(filtered_json)
    temp_df['timestamp'] = datetime.now().timestamp()


In [93]:
list_of_keys = ['id', 'symbol', 'name', 'hashing_algorithm', 'categories', 'genesis_date',  'sentiment_votes_up_percentage',
                'sentiment_votes_down_percentage', 'market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score',
                'community_score', 'liquidity_score', 'public_interest_score', 'last_updated']

In [94]:
output = {key: res[key] for key in list_of_keys}

In [14]:
session = boto3.Session(profile_name='master')

In [16]:
import awswrangler as wr
wr.catalog.databases(boto3_session=session)

,Database,Description
0,master_project,this is a test database


In [18]:
client = session.client('glue')

In [19]:
client.get_databases()

{'DatabaseList': [{'Name': 'master_project',
   'Description': 'this is a test database',
   'CreateTime': datetime.datetime(2022, 4, 5, 13, 16, 48, tzinfo=tzlocal()),
   'CreateTableDefaultPermissions': [{'Principal': {'DataLakePrincipalIdentifier': 'IAM_ALLOWED_PRINCIPALS'},
     'Permissions': ['ALL']}],
   'CatalogId': '142327076576'}],
 'ResponseMetadata': {'RequestId': '860e955b-1077-406e-8347-1c0b6c746e92',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 05 Apr 2022 11:26:50 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '297',
   'connection': 'keep-alive',
   'x-amzn-requestid': '860e955b-1077-406e-8347-1c0b6c746e92'},
  'RetryAttempts': 0}}